In [1]:
from features import ensembleFeature
import pandas as pd
import numpy as np
from utils import metricsCal
from thundersvm import SVC
from sklearn.model_selection import KFold

In [2]:
##将序列转化为词向量用到的函数
def fa_seq(filepath):
    f = open(filepath,'r')
    x = []
    for i in f:
        x.append(i)
    seq = []
    for i in range(len(x)):
        if i%2==1:
            seq.append(x[i][:-1])
    return seq

In [3]:
train_seq = fa_seq("data/Mouse/mouse_train.fasta")
trainLabel = np.append(np.ones(int(len(train_seq)/2)),np.zeros(int(len(train_seq)-len(train_seq)/2)),axis=0)
trainData_EIIP = ensembleFeature.EIIP(train_seq)

trainData_PseEIIP = ensembleFeature.PseEIIP(train_seq)

trainData_PCP = ensembleFeature.PCP(train_seq)

trainData_NCPA = ensembleFeature.NCPA(train_seq)
trainData_NCPA = trainData_NCPA.reshape(trainData_NCPA.shape[0],-1)

trainData_DBPF = ensembleFeature.DBPF(train_seq)
trainData_DBPF = trainData_DBPF.reshape(trainData_DBPF.shape[0],-1)

trainData = np.concatenate(  (trainData_EIIP,trainData_PseEIIP,trainData_PCP,trainData_NCPA,trainData_DBPF),axis=1)

In [4]:
from sklearn.decomposition import FastICA
ica = FastICA(n_components= 100).fit(trainData)

In [5]:
x = ica.transform(trainData)

In [10]:
#交叉验证
rows = 10
row = -1
row_name = []
Sen = np.zeros(rows)
Spe = np.zeros(rows)
Acc = np.zeros(rows)
Mcc = np.zeros(rows)
Auc = np.zeros(rows)

kf = KFold(10,True,10)
for i,[train_index, test_index] in enumerate(kf.split(X_reduced)):
    row+=1
    row_one = "Bert6mA_origin_KFold_" + str(i)
    row_name.append(row_one)
    X_train = X_reduced[train_index]
    X_test = X_reduced[test_index]
    Y_train = trainLabel[train_index]
    Y_test = trainLabel[test_index]
    
    model = SVC(C=1.0,gamma="auto")
    
    model.fit(X_train,Y_train)
    th,_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics( model.decision_function(X_train).reshape(-1), Y_train )
    _,_,_,_,sen,spe,acc,mcc,auc = metricsCal.get_test_metrics( model.decision_function(X_test), Y_test,th )
    Sen[row] = sen
    Spe[row] = spe
    Acc[row] = acc
    Mcc[row] = mcc
    Auc[row] = auc
    
All = np.vstack((Sen,Spe,Acc,Mcc,Auc))
np.mean(All[4])

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.7017765699617384

In [8]:
from sklearn.manifold import LocallyLinearEmbedding
lle = LocallyLinearEmbedding(n_components=100, n_neighbors=50, random_state=42)
X_reduced = lle.fit_transform(trainData)

In [9]:
X_reduced.shape

(9126, 100)

In [11]:
def get_feature_importances(data,label, shuffle, seed=None):
    # Gather real features
    train_features = [f for f in data if f not in ['TARGET', 'SK_ID_CURR']]
    # Go over fold and keep track of CV score (train and valid) and feature importances
    
    # Shuffle target if required
     y = data['TARGET'].copy()
    y=label
    if shuffle:
        # Here you could as well use a binomial distribution
        y = data['TARGET'].copy().sample(frac=1.0)
#        y = label.copy().sample(frac=1.0)
    
    # Fit LightGBM in RF mode, yes it's quicker than sklearn RandomForest
    dtrain = lgb.Dataset(data[train_features], y, free_raw_data=False, silent=True)
    lgb_params = {
        'objective': 'binary',
        'boosting_type': 'rf',
        'subsample': 0.623,
        'colsample_bytree': 0.7,
        'num_leaves': 127,
        'max_depth': 8,
        'seed': seed,
        'bagging_freq': 1,
        'n_jobs': 4
    }
    
    # Fit the model
    clf = lgb.train(params=lgb_params, train_set=dtrain, num_boost_round=200, categorical_feature=categorical_feats)
    # Get feature importances
    imp_df = pd.DataFrame()
    imp_df["feature"] = list(train_features)
    imp_df["importance_gain"] = clf.feature_importance(importance_type='gain')
    imp_df["importance_split"] = clf.feature_importance(importance_type='split')
    imp_df['trn_score'] = roc_auc_score(y, clf.predict(data[train_features]))
    
    return imp_df

In [33]:
# Seed the unexpected randomness of this world
np.random.seed(123)
header =  [f'fea_{i-1}' for i in range(len(trainData[0])+1)]
header[0] = ["TARGET"]
data = pd.DataFrame(data =  np.append( np.array(trainLabel).reshape(-1,1),trainData,axis=1),columns=header  )
# Get the actual importance, i.e. without shuffling
actual_imp_df = get_feature_importances(data, shuffle=False) 

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


KeyError: 'TARGET'

In [ ]:
null_imp_df = pd.DataFrame()
nb_runs = 10
import time
start = time.time()
dsp = ''
for i in range(nb_runs):
    # Get current run importances
    imp_df = get_feature_importances(data=data, shuffle=True)
    imp_df['run'] = i + 1 
    # Concat the latest importances with the old ones
    null_imp_df = pd.concat([null_imp_df, imp_df], axis=0)
    # Erase previous message
    for l in range(len(dsp)):
        print('\b', end='', flush=True)
    # Display current run and time used
    spent = (time.time() - start) / 60
    dsp = 'Done with %4d of %4d (Spent %5.1f min)' % (i + 1, nb_runs, spent)
    print(dsp, end='', flush=True)

In [36]:
data = pd.DataFrame(data =  np.append( np.array(trainLabel).reshape(-1,1),trainData,axis=1),columns=header  )

In [37]:
actual_imp_df = get_feature_importances(data, shuffle=False)

KeyError: 'TARGET'